In [1]:
import matplotlib.pyplot as plt
import numpy as np
import dask.array as da
import zarr as zr
import django

django.setup()
from elements import models, managers
import larvik.managers
import importlib
import xarray as xr
sample = models.Sample.objects.last()
from django.contrib.auth import get_user_model

2020-03-30 17:34:47,061 [mandal.settings] INFO     Hosting on ['*']
2020-03-30 17:34:47,063 [mandal.settings] INFO     Zarr Compression: None
2020-03-30 17:34:47,064 [mandal.settings] INFO     Zarr Dtype: <class 'float'>
2020-03-30 17:34:47,064 [mandal.settings] INFO     Redis Host: redis
2020-03-30 17:34:47,065 [mandal.settings] INFO     Redis Port: 6379
2020-03-30 17:34:47,066 [mandal.settings] INFO     Dask Scheduler Port: 5432
2020-03-30 17:34:47,067 [mandal.settings] INFO     Dask Scheduler Host: localhost
2020-03-30 17:34:47,068 [mandal.settings] INFO     Postgres Port: 5432
2020-03-30 17:34:47,070 [mandal.settings] INFO     Postgres Host: postgres
2020-03-30 17:34:47,071 [mandal.settings] INFO     Postgres User: hello_django
2020-03-30 17:34:47,072 [mandal.settings] INFO     Postgres DB: hello_django_dev
2020-03-30 17:34:47,073 [mandal.settings] INFO     Postgres Password: hello_django


In [2]:
lala = xr.DataArray(da.zeros((1024,1024,56,1,1)), dims=["x","y","c","z","t"])
lala = xr.DataArray(lala, coords={ "x": lala.x, "y": lala.y, "z": lala.z, "t": lala.t, "c": lala.c})

it = models.Representation.objects.from_xarray(lala,sample=sample,creator=get_user_model().objects.last())

2020-03-30 17:35:00,406 [larvik.stores] INFO     Folder [/code/media]: Connecting to 2-sample.representation.testL169ZH


In [3]:
it.store.load()

2020-03-30 17:35:23,724 [larvik.stores] INFO     Folder [/code/media]: Connecting to 2-sample.representation.testL169ZH


<xarray.DataArray 'data' (x: 1024, y: 1024, c: 56, z: 1, t: 1)>
dask.array<zarr, shape=(1024, 1024, 56, 1, 1), dtype=float64, chunksize=(128, 128, 14, 1, 1), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 0 1 2 3 4 5 6 7 8 9 10 ... 46 47 48 49 50 51 52 53 54 55
  * t        (t) int64 0
  * x        (x) int64 0 1 2 3 4 5 6 7 ... 1017 1018 1019 1020 1021 1022 1023
  * y        (y) int64 0 1 2 3 4 5 6 7 ... 1017 1018 1019 1020 1021 1022 1023
  * z        (z) int64 0

In [4]:
it.store.url

'/images/2-sample.representation.testL169ZH'

In [6]:
from larvik.fields import XArrayStore
from larvik.standardstorage import ZarrStorage
from django.core.files import File

In [7]:

from storages.backends.s3boto3 import S3Boto3StorageFile

In [8]:
S3Boto3StorageFile("1-sample/representation-group/test", "w", ZarrStorage())

<S3Boto3StorageFile: 1-sample/representation-group/test>

In [11]:
it.store = XArrayStore(S3Boto3StorageFile, it.store, "1-sample/representation-group/test")

In [23]:
store = it.store.connect

2020-03-30 14:59:15,503 [larvik.fields] INFO     Connecting to zarr/1-sample/representation-group/test


In [24]:
lala.to_dataset(name="data").to_zarr(store=store, mode="w", compute="True")

In [25]:
zarr_ds = xr.open_zarr(store=store, consolidated=False)

In [15]:
zarr_ds

<xarray.Dataset>
Dimensions:  (c: 1, t: 1, x: 1024, y: 1024, z: 1)
Coordinates:
  * c        (c) int64 0
  * t        (t) int64 0
  * x        (x) int64 0 1 2 3 4 5 6 7 ... 1017 1018 1019 1020 1021 1022 1023
  * y        (y) int64 0 1 2 3 4 5 6 7 ... 1017 1018 1019 1020 1021 1022 1023
  * z        (z) int64 0
Data variables:
    data     (x, y, c, z, t) float64 dask.array<chunksize=(256, 256, 1, 1, 1), meta=np.ndarray>

In [16]:
import s3fs
s3_path = f"zarr/1-sample/representation-group/test"
# Initilize the S3 file system
s3 = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "http://minio:9000"})
store = s3fs.S3Map(root=s3_path, s3=s3, check=False)

In [17]:
zarr_ds = xr.open_zarr(store=store, consolidated=False)

ValueError: group not found at path ''

In [44]:
class TemporaryFile(object): 
    def __init__(self, field, tmp="/tmp"): 
        self.field = field
        self.tmppath = tmp
      
    def __enter__(self): 
        import os
        import uuid
        _, file_extension = os.path.splitext(self.field.name)
        self.filename = self.tmppath + "/" + str(uuid.uuid4()) + file_extension
        with open(self.filename, 'wb+') as destination:
            for chunk in self.field.chunks():
                destination.write(chunk)

        return self.filename
    
    def __exit__(self, exc_type, exc_value, tb):
        import os
        os.remove(self.filename)
        if exc_type is not None:
            traceback.print_exception(exc_type, exc_value, tb)
            # return False # uncomment to pass exception through

        return True

In [45]:
with Temporary(bioimage.file,tmp="/tmp") as tmpfile:
    print(tmpfile)
    
    
    
    
    
    

/tmp/4532b8ee-8469-4da5-80b8-05108b8006bc.nd2


In [27]:
nana

'/tmp/a019d979-19ef-4e8d-b839-048bd772205f.nd2'

In [12]:
with open(bioimage.file.name, 'wb+') as destination:
        for chunk in bioimage.file.chunks():
            destination.write(chunk)